In [1]:
from linares_plot import *
import os
import datetime
from datetime import datetime as dt
import re
import statsmodels.formula.api as smf

In [2]:
##Actions Gea data
df_gea=pd.read_excel(os.getcwd() + '\\EVENTOS_GEA.xlsx', sheet_name='Hoja2' )
times_s = [df_gea.tiempo.iloc[i].minute*60 + df_gea.tiempo.iloc[i].second for i in range(len(df_gea))]
df_gea['times_s'] = times_s

### cambiar tiempo y tiempo_s por los corregidos, en los que se restan 10 minutos, 31 segundos. De esta manera, 
### los dos dataframes están sincronizados

new_times=[]
times_s_new=[]

for i in range(len(df_gea)):
    #print(i)
    h = df_gea.tiempo.iloc[i]
    h_t = str(h.hour) + ':' + str(h.minute) + ':' + str(h.second) 
    substracting = '00:10:31'
    new_time = dt.strptime(h_t, "%H:%M:%S") - dt.strptime(substracting, "%H:%M:%S")
    new_times.append(new_time)
    ##
    times_s_new.append(df_gea.times_s.loc[i] - (60*10 + 31) ) 

    
df_gea['tiempo'] = new_times 
df_gea['times_s'] = times_s_new 

df_gea = df_gea.sort_values(by=['times_s'])

df_gea= df_gea.reset_index()
df_gea = df_gea[['tiempo', 'evento', 'decripcion', 'times_s']]
df_gea['evento_'] = df_gea['evento'].copy()
df_gea['evento_'] = df_gea['evento_'].replace(['Miniclimax', 'Climax final' ], ['climax', 'climax']) #for simplicity
df_gea.head()

,tiempo,evento,decripcion,times_s,evento_
0,00:00:27,gag,broma sobre su dominio del catalán,27,gag
1,00:00:34,gag,"""Por deferencia""",34,gag
2,00:00:46,inicio,Abre el estuche de la baraja,46,inicio
3,00:01:05,gag,"""Que haya público""",65,gag
4,00:01:19,gag,"""El público de schroedinger""",79,gag


In [3]:
##Memory data
df_ = pd.read_excel('C:\\Users\\David\\Documents\\GitHub\\Gea_memory\\df_proces.xlsx') 

df_int = df_.loc[(df_['session']==1) & ((df_['juego']==1) | (df_['juego']==2) | (df_['juego']==3)  | 
                               (df_['juego']==4)  | (df_['juego']==5)  | (df_['juego']==6)  | 
                               (df_['juego']==7) | (df_['juego']==8)), 
        ['recuerdo', 'juego', 'asiento', 'subj', 'tiempo', 'n_saliencias', 'edad', 'sexo']]

df_int = df_int.reset_index(drop=True) 
df_int['subj']=df_int['subj']+1
df_int.head()

,recuerdo,juego,asiento,subj,tiempo,n_saliencias,edad,sexo
0,0,1,38,1,1,0,24,m
1,0,2,38,1,1,0,24,m
2,0,3,38,1,1,0,24,m
3,1,4,38,1,1,1,24,m
4,1,5,38,1,1,3,24,m


## Procesado de datos  
Hasta ahora se hace en cada truco. Cuanod los excels estén correctos, las siguientes cells se pueden poner en un loop y se pondrán todos los trucos juntos.  
***en el 5 hay muchas A sin label, les he puetso 1  
faltan la gran mayoria del 6, les he puesto 1***

In [4]:
frames=[]

trick_list=['Truco 1','Truco 2','Truco 3','Truco 4','Truco 5','Truco 6','Truco 7','Truco 8']

for T in trick_list:
    print(T)
    Df=pd.read_excel(os.getcwd() + '\\ANOTACIONES2.xlsx', sheet_name= T )
    list_columns_subjects = list(Df.columns)
    list_columns_subjects.remove('tiempo')
    for subject in list_columns_subjects:
        #print(subject)
        dfs = Df[['tiempo', subject]]
        dfs = dfs.loc[~pd.isna(dfs[subject])] # Quitar todos los NaN (qudarnos solo con los eventos)
        ####################################### Añadir una columna de segundos
        times_s=[]
        for i in range(0, len(dfs)):
            l = dfs.tiempo.iloc[i]
            times_s.append(  int(datetime.timedelta(hours=l.hour, minutes=l.minute, seconds=l.second).total_seconds()) )
        #
        dfs.columns=['tiempo', 'event']
        dfs['time_s'] = times_s
        ####################################### Añadir columna de subject y de truco
        dfs['trick'] = int(T.split(' ')[-1]) ## last character of the string is the number of the trick
        dfs['subject'] = int(re.findall('\d+', subject)[0]) ##separate leter from number
        frames.append(dfs)

    
dfs=pd.concat(frames)
dfs.head()

Truco 1
Truco 2
Truco 3
Truco 4
Truco 5
Truco 6
Truco 7
Truco 8


,tiempo,event,time_s,trick,subject
3,00:00:06,Se,6,1,1
5,00:00:10,Si,10,1,1
6,00:00:12,N,12,1,1
16,00:00:32,In,32,1,1
31,00:01:02,In,62,1,1


In [5]:
dfs['event'] = dfs['event'].replace(['L '], ['L'])
dfs['event'] = dfs['event'].replace(['L  '], ['L'])
dfs['event'] = dfs['event'].replace(['Si '], ['Si'])
dfs['event'] = dfs['event'].replace(['SI'], ['Si'])
dfs['event'] = dfs['event'].replace(['O '], ['O'])
dfs['event'] = dfs['event'].replace(['O  '], ['O'])
dfs['event'] = dfs['event'].replace(['In '], ['In'])
dfs['event'] = dfs['event'].replace(['In  '], ['In'])
dfs['event'] = dfs['event'].replace(['In   '], ['In'])
dfs['event'] = dfs['event'].replace(['IN'], ['In'])
dfs['event'] = dfs['event'].replace(['I  '], ['I'])
dfs['event'] = dfs['event'].replace(['I   '], ['I'])
dfs['event'] = dfs['event'].replace(['C '], ['C'])
dfs['event'] = dfs['event'].replace(['  C'], ['C'])
dfs['event'] = dfs['event'].replace(['In '], ['In'])
dfs['event'] = dfs['event'].replace(['N '], ['N'])
dfs['event'] = dfs['event'].replace(['N  '], ['N'])
dfs['event'] = dfs['event'].replace(['N* '], ['N*'])
dfs['event'] = dfs['event'].replace(['N*  '], ['N*'])
dfs['event'] = dfs['event'].replace(['VP '], ['VP'])
dfs['event'] = dfs['event'].replace(['PV'], ['VP'])
dfs['event'] = dfs['event'].replace(['S '], ['S'])
dfs['event'] = dfs['event'].replace(['S  '], ['S'])
dfs['event'] = dfs['event'].replace(['F '], ['F'])
dfs['event'] = dfs['event'].replace(['F  '], ['F'])
dfs['event'] = dfs['event'].replace(['P '], ['P'])
dfs['event'] = dfs['event'].replace(['p'], ['P'])
dfs['event'] = dfs['event'].replace(['p*'], ['P*'])
dfs['event'] = dfs['event'].replace(['M '], ['M'])
dfs['event'] = dfs['event'].replace(['M  '], ['M'])
dfs['event'] = dfs['event'].replace(['R '], ['R'])
dfs['event'] = dfs['event'].replace(['A 1'], ['A1'])

In [6]:
df_subjects=[]

for subject in dfs.subject.unique():
    #print(subject)
    ####################################################### get the subject and sort by time, add duration and reset index
    df_x = dfs.loc[dfs['subject']==subject]
    df_x = df_x.sort_values(by=['time_s'])
    df_x['duration']=1
    df_x = df_x.reset_index(drop=True) 
    ###################################################### get actions with * --> calculate durations of previous and eliminate * actions
    boole=[] ###bolean to get the final of actions
    for i in range(len(df_x)):
        stri = df_x['event'].iloc[i]
        boole.append( stri[-1]=='*' )
    #
    boole=np.array(boole)
    finals = df_x[boole]
    for fi in range(len(finals)):
        event_final = finals.event.iloc[fi][:-1] ##without the astherisc
        time_final = finals.time_s.iloc[fi]
        #
        previous_event = df_x.loc[(df_x['event']==event_final) & (df_x['time_s']<time_final) ].iloc[-1] ##get the last one
        duration_event = time_final - previous_event.time_s
        ## Change the duration of the dataframe
        df_x.at[previous_event.name, 'duration'] = duration_event
    #
    df_x = df_x[~boole] ### remove the events with astherist
    ######################################################  gat applauses --> add duration (break A)
    boole_apl=[] ###bolean to get the final of actions
    for i in range(len(df_x)):
        stri = df_x['event'].iloc[i]
        if stri[0]=='a':
            boole_apl.append(True)
        elif stri[0]=='A':
            boole_apl.append(True)
        else:
            boole_apl.append(False)
    #
    boole_apl=np.array(boole_apl)    
    aplauses = df_x[boole_apl]
    for ap in range(len(aplauses)):
        event_ = aplauses.iloc[ap] 
        name_event = event_.event
        duration_app = int(re.findall('\d+', name_event)[0])
        evnt = re.split('(\d+)', name_event)[0]
        #
        df_x.at[event_.name, 'duration'] = duration_app
        df_x.at[event_.name, 'event'] = evnt
    ###
    df_subjects.append(df_x)
    #
#



df_subjects=pd.concat(df_subjects)
df_subjects.head()

,tiempo,event,time_s,trick,subject,duration
0,00:00:06,Se,6,1,1,1
1,00:00:10,Si,10,1,1,1
2,00:00:12,N,12,1,1,102
3,00:00:32,In,32,1,1,1
4,00:01:02,In,62,1,1,1


In [7]:
### Anotaciones de risa y sonrisa
df_subjects_risa=[]

for T in trick_list:
    df_risa = pd.read_excel(os.getcwd() + '\\ANOTACIONES3.xlsx', sheet_name= T )

    list_columns_subjects = list(df_risa.columns)
    list_columns_subjects.remove('Tiempo')
    list_columns_subjects=list_columns_subjects ## para ir comprobando, una a una, que no haya espacios vacíos!    

    for COL in list_columns_subjects:
        event_type = COL[0]
        subject = int(re.findall('\d+', COL)[0])
        df_x=df_risa[['Tiempo', COL]]
        times_s=[]
        for i in range(0, len(df_x)):
            l = df_x.Tiempo.iloc[i]
            times_s.append( int(datetime.timedelta(hours=l.hour, minutes=l.minute, seconds=l.second).total_seconds())  )
        #
        df_x.columns=['tiempo', 'event']
        df_x['time_s'] = times_s
        ##########################################################
        #### get starts and durations
        counter=0
        starts=[]
        finals=[]
        for i in range(0, len(df_x)):
            if df_x.iloc[i, 1]==0:
                if counter==0:
                    pass
                else:
                    finals.append([df_x.loc[i, 'tiempo'], df_x.loc[i, 'event'], df_x.loc[i, 'time_s']])
                    counter=0
            elif df_x.iloc[i, 1]==1:
                if counter==0:
                    starts.append([df_x.loc[i, 'tiempo'], df_x.loc[i, 'event'], df_x.loc[i, 'time_s']])
                counter+=1
            ###
            if i==len(df_x)-1: ##si al final todavia están
                if counter !=0:
                    finals.append([df_x.loc[i, 'tiempo'], df_x.loc[i, 'event'], df_x.loc[i, 'time_s']])
        ###
        ###
        if len(starts)>0:
            durations = np.array(finals)[:, -1] - np.array(starts)[:, -1]
            ###
            df_evnt = pd.DataFrame(starts)
            df_evnt.columns=['tiempo', 'event', 'time_s']
            df_evnt['event'] = event_type
            df_evnt['duration'] = durations
            df_evnt['subject'] = subject
            df_evnt['trick']= int(T.split(' ')[-1]) 
            df_subjects_risa.append(df_evnt)
            #print('append')
        #else:
            #print('nothing')


        
df_subjects_risa=pd.concat(df_subjects_risa)
df_subjects_risa.head()

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,tiempo,event,time_s,duration,subject,trick
0,00:02:20,S,140,12,1,1
1,00:02:36,S,156,2,1,1
2,00:02:46,S,166,2,1,1
3,00:03:18,S,198,4,1,1
4,00:03:48,S,228,6,1,1


In [8]:
##cambiando los S y los R de anotaciones2 por los de anotaciones3
df_subjects2 = df_subjects.reset_index(drop=True) 
df1 = df_subjects2.drop(df_subjects2[(df_subjects2['event']=='R') | (df_subjects2['event']=='S') ].index)
df_comb = pd.concat([df1, df_subjects_risa])
df_comb = df_comb.sort_values(by=['subject', 'trick', 'time_s']) #primero por subject y, dentro de subject, por tiempo
df_comb = df_comb.reset_index(drop=True) 

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [9]:
### añadir los eventos previos y posteriores y la memoria

df_subjects_comb=[]

for subject in df_comb.subject.unique():
    print(subject)
    for trick_ in list(np.arange(1,9)):
        #
        df_x=df_comb.loc[(df_comb['subject']==subject) & (df_comb['trick']==trick_) ]
        ###################################################### get previous action of the magician and Reaction time respect it
        prev_event=[]
        times_after=[]
        for evnt in range(len(df_x)):
            time_ = df_x.iloc[evnt].time_s    
            previous_events_gea = df_gea.loc[df_gea['times_s']<=time_]
            if len(previous_events_gea)==0:
                prev_event.append('None')
                times_after.append(-1)
            else:
                previous_event=previous_events_gea.evento.iloc[-1]
                prev_event.append(previous_event)
                time_after = df_x.time_s.iloc[evnt] - previous_events_gea.times_s.iloc[-1]
                times_after.append(time_after)
        ##
        df_x['RT'] = times_after
        df_x['previous_event'] = prev_event
        ###################################################### get future action of the magician and anticiipation time respect it
        next_event=[]
        times_before=[]
        for evnt in range(len(df_x)): 
            time_ = df_x.iloc[evnt].time_s    
            posts_events_gea = df_gea.loc[df_gea['times_s']>=time_]
            if len(posts_events_gea)==0:
                next_event.append('None')
                times_before.append(1)
            else:
                next_evnt=posts_events_gea.evento.iloc[0]
                next_event.append(next_evnt)
                time_before = df_x.time_s.iloc[evnt] - posts_events_gea.times_s.iloc[0]
                times_before.append(time_before)
        ##
        df_x['anticip_time'] = times_before
        df_x['next_event'] = next_event
        ###################################################### add memory data
        ###
        if len(df_int.loc[(df_int['subj']==subject) & (df_int['juego']==trick_) & (df_int['tiempo']==1)]):
            r1 = df_int.loc[(df_int['subj']==subject) & (df_int['juego']==trick_) & (df_int['tiempo']==1)].recuerdo.iloc[0]
        else:
            r1='Miss'

        if len(df_int.loc[(df_int['subj']==subject) & (df_int['juego']==trick_) & (df_int['tiempo']==2)]):
            r2 = df_int.loc[(df_int['subj']==subject) & (df_int['juego']==trick_) & (df_int['tiempo']==2)].recuerdo.iloc[0]
        else:
            r2='Miss'

        if len(df_int.loc[(df_int['subj']==subject) & (df_int['juego']==trick_) & (df_int['tiempo']==3)]):
            r3 = df_int.loc[(df_int['subj']==subject) & (df_int['juego']==trick_) & (df_int['tiempo']==3)].recuerdo.iloc[0]
        else:
            r3='Miss'

        if len(df_int.loc[(df_int['subj']==subject) & (df_int['juego']==trick_) & (df_int['tiempo']==4)]):
            r4 = df_int.loc[(df_int['subj']==subject) & (df_int['juego']==trick_) & (df_int['tiempo']==4)].recuerdo.iloc[0]
        else:
            r4='Miss'

        df_x['r1'] = r1
        df_x['r2'] = r2
        df_x['r3'] = r3
        df_x['r4'] = r4
        ######################################################

        df_subjects_comb.append(df_x)

C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


1


C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\David\Anaconda3\envs\python3\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [10]:
df=pd.concat(df_subjects_comb)
df.head()

,duration,event,subject,tiempo,time_s,trick,RT,previous_event,anticip_time,next_event,r1,r2,r3,r4
0,1,Se,1,00:00:06,6,1,-1,None,-21,gag,0,0,0,0
1,1,Si,1,00:00:10,10,1,-1,None,-17,gag,0,0,0,0
2,102,N,1,00:00:12,12,1,-1,None,-15,gag,0,0,0,0
3,1,In,1,00:00:32,32,1,5,gag,-2,gag,0,0,0,0
4,1,In,1,00:01:02,62,1,16,inicio,-3,gag,0,0,0,0


In [11]:
len(df)

11687

In [13]:
### df.to_excel('C:\\Users\\David\\Desktop\\David\\risa_aplausos\\df_subjects.xlsx')